In [1]:
import pandas as pd
import numpy as np

df_cigatette = pd.read_csv('cigarette.csv', index_col=0)
df_cigatette.head()

,state,year,cpi,pop,packpc,income,tax,avgprs,taxs
1,AL,1985,1.076,3973000,116.486282,46014968,32.500004,102.181671,33.348335
2,AR,1985,1.076,2327000,128.534592,26210736,37.000000,101.474998,37.000000
3,AZ,1985,1.076,3184000,104.522614,43956936,31.000000,108.578751,36.170418
4,CA,1985,1.076,26444000,100.363037,447102816,26.000000,107.837341,32.104000
5,CO,1985,1.076,3209000,112.963539,49466672,31.000000,94.266663,31.000000


In [2]:
df_cigatette['packpc'].describe()

count    528.000000
mean     106.454967
std       23.124227
min       49.272198
25%       91.405214
50%      105.909554
75%      119.592928
max      197.993988
Name: packpc, dtype: float64

In [3]:
dict_states = {
    'AL': ['Alabama', '亚拉巴马州'],
    'AK': ['Alaska', '阿拉斯加州'],
    'AZ': ['Arizona', '亚利桑那州'],
    'AR': ['Arkansas', '阿肯色州'],
    'CA': ['California', '加利福尼亚州'],
    'CO': ['Colorado', '科罗拉多州'],
    'CT': ['Connecticut', '康涅狄格州'],
    'DE': ['Delaware', '特拉华州'],
    'FL': ['Florida', '佛罗里达州'],
    'GA': ['Georgia', '佐治亚州'],
    'HI': ['Hawaii', '夏威夷州'],
    'ID': ['Idaho', '爱达荷州'],
    'IL': ['Illinois', '伊利诺伊州'],
    'IN': ['Indiana', '印第安纳州'],
    'IA': ['Iowa', '艾奥瓦州'],
    'KS': ['Kansas', '堪萨斯州'],
    'KY': ['Kentucky', '肯塔基州'],
    'LA': ['Louisiana', '路易斯安那州'],
    'ME': ['Maine', '缅因州'],
    'MD': ['Maryland', '马里兰州'],
    'MA': ['Massachusetts', '马萨诸塞州'],
    'MI': ['Michigan', '密歇根州'],
    'MN': ['Minnesota', '明尼苏达州'],
    'MS': ['Mississippi', '密西西比州'],
    'MO': ['Missouri', '密苏里州'],
    'MT': ['Montana', '蒙大拿州'],
    'NE': ['Nebraska', '内布拉斯加州'],
    'NV': ['Nevada', '内华达州'],
    'NH': ['New Hampshire', '新罕布什尔州'],
    'NJ': ['New Jersey', '新泽西州'],
    'NM': ['New Mexico', '新墨西哥州'],
    'NY': ['New York', '纽约州'],
    'NC': ['North Carolina', '北卡罗来纳州'],
    'ND': ['North Dakota', '北达科他州'],
    'OH': ['Ohio', '俄亥俄州'],
    'OK': ['Oklahoma', '俄克拉何马州'],
    'OR': ['Oregon', '俄勒冈州'],
    'PA': ['Pennsylvania', '宾夕法尼亚州'],
    'RI': ['Rhode Island', '罗得岛州'],
    'SC': ['South Carolina', '南卡罗来纳州'],
    'SD': ['South Dakota', '南达科他州'],
    'TN': ['Tennessee', '田纳西州'],
    'TX': ['Texas', '得克萨斯州'],
    'UT': ['Utah', '犹他州'],
    'VT': ['Vermont', '佛蒙特州'],
    'VA': ['Virginia', '弗吉尼亚州'],
    'WA': ['Washington', '华盛顿州'],
    'WV': ['West Virginia', '西弗吉尼亚州'],
    'WI': ['Wisconsin', '威斯康星州'],
    'WY': ['Wyoming', '怀俄明州']
}
df_cigatette['state_fn'] = df_cigatette['state'].apply(lambda x: dict_states[x][0])
df_cigatette['year_str'] = df_cigatette['year'].astype(str)

df_cigatette.head()

,state,year,cpi,pop,packpc,income,tax,avgprs,taxs,state_fn,year_str
1,AL,1985,1.076,3973000,116.486282,46014968,32.500004,102.181671,33.348335,Alabama,1985
2,AR,1985,1.076,2327000,128.534592,26210736,37.000000,101.474998,37.000000,Arkansas,1985
3,AZ,1985,1.076,3184000,104.522614,43956936,31.000000,108.578751,36.170418,Arizona,1985
4,CA,1985,1.076,26444000,100.363037,447102816,26.000000,107.837341,32.104000,California,1985
5,CO,1985,1.076,3209000,112.963539,49466672,31.000000,94.266663,31.000000,Colorado,1985


In [4]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [5]:
def generate_box_plot_data(df, category, column):

    # find the quartiles and IQR for each category
    groups = df[[column, category]].groupby(by=category, sort=False)
    # generate some synthetic time series for six different categories
    cats = []

    q1 = groups.quantile(q=0.25)
    q2 = groups.quantile(q=0.5)
    q3 = groups.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr
    # find the outliers for each category
    def outliers(group):
        cat = group.name
        cats.append(cat)
        return group[(group[column] > upper.loc[cat][column]) | (group[column] < lower.loc[cat][column])][column]
    out = groups.apply(outliers).dropna()
    # prepare outlier data for plotting, we need coordinates for every outlier.
    if not out.empty:
        out_index = []
        out_category = []
        out_value = []
        for keys in out.index:
            out_index.append(keys[1])
            out_category.append(keys[0])
            out_value.append(out.loc[keys[0]].loc[keys[1]])

    # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)

    upper[column] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,column]),upper[column])]
    lower[column] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,column]),lower[column])]

    data_box_plot = {
        'category': cats,
        'qmin': qmin[column].tolist(),
        'q1' : q1[column].tolist(),
        'q2' : q2[column].tolist(),
        'q3' : q3[column].tolist(),
        'iqr': iqr[column].tolist(),
        'qmax': qmax[column].tolist(),
        'upper': upper[column].tolist(),
        'lower': lower[column].tolist()
    }

    data_box_plot_outlier = {
        'index': out_index,
        'category' : out_category,
        'value': out_value
    }
    
    return data_box_plot, data_box_plot_outlier

def bokeh_box_plot(p, data_box_plot, data_box_plot_out, all_dot=False):
    box, out = data_box_plot, data_box_plot_out
    box = ColumnDataSource(data=box)
    # stems
    p.segment(x0='upper', y0='category', x1='q3', y1='category', line_color='black', source=box)
    p.segment(x0='lower', y0='category', x1='q1', y1='category', line_color='black', source=box)

    # boxes
    p.hbar(y='category', height=0.7, right='q2', left='q3', fill_color='navy', alpha=0.5, line_color='black', source=box)
    p.hbar(y='category', height=0.7, right='q1', left='q2', fill_color='navy', alpha=0.3, line_color='black', source=box)

    # whiskers (almost-0 height rects simpler than segments)
    p.rect(x='lower', y='category', width=0.01, height=0.2, line_color='black', source=box)
    p.rect(x='upper', y='category', width=0.01, height=0.2, line_color='black', source=box)

    # outliers
    p.circle(x='value', y='category', size=6, color="#F38630", fill_alpha=1, source=out)
    
    return box


In [6]:
class box_plot_app_by_year():
    def __init__(self, df):
        self.df = df.copy()
        self.df['year_str'] = self.df['year'].astype(str)
        self.df.sort_values(by='year', inplace=True, ascending=False)
        self.data_box_plot, self.data_box_plot_out = generate_box_plot_data(df_cigatette, 'year_str', 'packpc')
        self.data_box_plot['category'] = [int(d) for d in self.data_box_plot['category']]
        self.data_box_plot_out['category'] = [int(d) for d in self.data_box_plot_out['category']]
        self.data_box_plot_out['text'] = self.df.loc[self.data_box_plot_out['index'], 'state'].tolist()

        p = figure(title='美国香烟消费 1985～1995', tools="save", 
                   # background_fill_color="#efefef", 
                   x_range=[45, 205], 
                   y_range=[1996, 1984],
                   plot_width=800, plot_height=450, 
                   toolbar_location='right', )
        self.data_source = bokeh_box_plot(p, self.data_box_plot, self.data_box_plot_out, all_dot=False)
        p.yaxis.axis_label = '<--- 年 --->'
        p.xaxis.axis_label = '香烟消费（平均每人每年的包数）'
        p.title.text_font_size = '16pt'
        p.axis.axis_label_text_font_size = "12pt"
        p.axis.major_label_text_font_size = "10pt"
        p.xaxis.ticker.min_interval = 50
        p.xaxis.ticker.num_minor_ticks = 5
        p.yaxis.ticker = [_ for _ in range(1995, 1984, -1)]
        p.text(x='value', y='category', text='text',
               text_font_size='10px', text_align='center', text_baseline='top', y_offset=5, 
               source=self.data_box_plot_out)
        p.line(x=[100, 100], y=['1996', '1984'], line_color='red', line_dash='dashed')
        self.plot_handler = show(p, notebook_handle=True)
        self.figure = p
        
box_plot_app_by_year(df_cigatette)

In [7]:
df_cigatette.sort_values(by='year', inplace=True, ascending=False)
data_box_plot, data_box_plot_out = generate_box_plot_data(df_cigatette, 'year_str', 'packpc')

#
data_box_plot['category'] = [int(d) for d in data_box_plot['category']]
data_box_plot_out['category'] = [int(d) for d in data_box_plot_out['category']]
#

p = figure(title='美国香烟消费 1985～1995', tools="save", 
           # background_fill_color="#efefef", 
           x_range=[45, 205], 
           y_range=[1996, 1984],
           plot_width=800, plot_height=450, 
           toolbar_location='right', )
p.yaxis.axis_label = '<--- 年 --->'
p.xaxis.axis_label = '香烟消费（平均每人每年的包数）'
p.title.text_font_size = '16pt'
p.axis.axis_label_text_font_size = "12pt"
p.axis.major_label_text_font_size = "10pt"
p.xaxis.ticker.min_interval = 50
p.xaxis.ticker.num_minor_ticks = 5
p.yaxis.ticker = [_ for _ in range(1995, 1984, -1)]

#     if all_dot:
#         p.circle(df[column], df[by], size=4, color="black", fill_alpha=0.3)
# p.xaxis.ticker = [50, 100, 150, 200]
# p.yaxis.ticker = ['1']
# p.xaxis.ticker.minor_ticks = [51]
# p.ygrid.grid_line_color = "grey"
# p.grid.grid_line_width = 0.5

data_box_plot_out['text'] = df_cigatette.loc[data_box_plot_out['index'], 'state'].tolist()
p.text(x='value', y='category', text='text',
       text_font_size='10px', text_align='center', text_baseline='top', y_offset=5, source=data_box_plot_out)
p.line(x=[100, 100], y=['1996', '1984'], line_color='red', line_dash='dashed')

bokeh_box_plot(p, data_box_plot, data_box_plot_out, all_dot=False)
show(p)

---

In [8]:
class box_plot_app_by_state():
    def __init__(self, df):
        self.df = df.copy()
        self.df.sort_values(by='state', inplace=True, ascending=True)
        self.data_box_plot, self.data_box_plot_out = generate_box_plot_data(df_cigatette, 'state_fn', 'packpc')
        self.df_box_plot = pd.DataFrame(self.data_box_plot)

        p = figure(title='美国香烟消费 1985～1995', tools="save", 
                   # background_fill_color="#efefef", 
                   x_range=[45, 205], 
                   y_range=self.data_box_plot['category'], 
                   plot_width=800, plot_height=1000, 
                   toolbar_location='right', )
        self.data_source = bokeh_box_plot(p, self.data_box_plot, self.data_box_plot_out, all_dot=False)
        p.yaxis.axis_label = ''
        p.xaxis.axis_label = '香烟消费（平均每人每年的包数）'
        p.title.text_font_size = '16pt'
        p.axis.axis_label_text_font_size = "12pt"
        p.axis.major_label_text_font_size = "10pt"
        p.xaxis.ticker.min_interval = 50
        p.xaxis.ticker.num_minor_ticks = 5
        
        self.plot_handler = show(p, notebook_handle=True)
        self.figure = p

    def update_app(self, sort_column):
        self.df_box_plot.sort_values(by=sort_column, inplace=True)
        self.data_box_plot = self.df_box_plot.to_dict('list')
        self.figure.y_range.factors = self.data_box_plot['category']
        push_notebook(handle=self.plot_handler)


In [9]:
import ipywidgets as widgets

sort_choise = widgets.Dropdown(
    options=['美国州', '香烟消费中值（包/人/年）', '香烟消费 IQR（包/人/年）'],
    index=0,
    description='排序方式：',
    disabled=False,
    layout={
        'width': '400px'
    },
    style={
        'description_width': 'initial'
    }   
)

display(sort_choise)

def on_selection_change(change):
    new_selection = change['new']
    index = change['owner'].index
    sort_column = ['category', 'q2', 'iqr'][index]
    app.update_app(sort_column)

sort_choise.observe(on_selection_change, names='value')
app = box_plot_app_by_state(df_cigatette)

Dropdown(description='排序方式：', layout=Layout(width='400px'), options=('美国州', '香烟消费中值（包/人/年）', '香烟消费 IQR（包/人/年）'…

In [10]:
df_cigatette.sort_values(by='state', inplace=True, ascending=True)
df_cigatette.head()

,state,year,cpi,pop,packpc,income,tax,avgprs,taxs,state_fn,year_str
1,AL,1985,1.076,3973000,116.486282,46014968,32.500004,102.181671,33.348335,Alabama,1985
241,AL,1990,1.307,4048508,111.744873,64094948,32.500004,143.448578,33.756920,Alabama,1990
193,AL,1989,1.240,4030000,109.205956,60044480,32.500004,133.256012,33.656002,Alabama,1989
433,AL,1994,1.482,4232965,103.426323,79832152,40.500004,156.275574,41.883919,Alabama,1994
49,AL,1986,1.096,3992000,117.159317,48703940,32.500004,107.989166,33.405838,Alabama,1986


In [11]:
data_box_plot, data_box_plot_out = generate_box_plot_data(df_cigatette, 'state_fn', 'packpc')

df_box_plot = pd.DataFrame(data_box_plot)
# df_box_plot.sort_values(by='q2').head()
# df_box_plot = pd.DataFrame(data_box_plot)
# df_box_plot.sort_values(by='q3', inplace=True)
data_box_plot = df_box_plot.to_dict('list')

p = figure(title='美国香烟消费 1985～1995', tools="save", 
           # background_fill_color="#efefef", 
           x_range=[45, 205], 
           y_range=data_box_plot['category'], 
           plot_width=800, plot_height=1000, 
           toolbar_location='right', )
p.yaxis.axis_label = '<--- 年 --->'
p.xaxis.axis_label = '香烟消费（平均每人每年的包数）'
p.title.text_font_size = '16pt'
p.axis.axis_label_text_font_size = "12pt"
p.axis.major_label_text_font_size = "10pt"
p.xaxis.ticker.min_interval = 50
p.xaxis.ticker.num_minor_ticks = 5
#     if all_dot:
#         p.circle(df[column], df[by], size=4, color="black", fill_alpha=0.3)
# p.xaxis.ticker = [50, 100, 150, 200]
# p.yaxis.ticker = ['1']
# p.xaxis.ticker.minor_ticks = [51]
# p.ygrid.grid_line_color = "grey"
# p.grid.grid_line_width = 0.5

bokeh_box_plot(p, data_box_plot, data_box_plot_out, all_dot=False)
show(p)

---

In [12]:
def bokeh_box_horizontal(p, df, by, column, direction=None, all_dot=False):

    # generate some synthetic time series for six different categories
    cats = list(df[by].unique())
    # find the quartiles and IQR for each category
    groups = df[[column, by]].groupby(by=by, sort=False)
    q1 = groups.quantile(q=0.25)
    q2 = groups.quantile(q=0.5)
    q3 = groups.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr
    # find the outliers for each category
    def outliers(group):
        cat = group.name
        return group[(group[column] > upper.loc[cat][column]) | (group[column] < lower.loc[cat][column])][column]
    out = groups.apply(outliers).dropna()
    # prepare outlier data for plotting, we need coordinates for every outlier.
    if not out.empty:
        outx = []
        outy = []
        for keys in out.index:
            outx.append(keys[0])
            outy.append(out.loc[keys[0]].loc[keys[1]])

    # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)

    upper[column] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,column]),upper[column])]
    lower[column] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,column]),lower[column])]

    if direction is None:
        direction = 'horizontal'

    if direction == 'horizontal':
        # stems
        p.segment(upper[column], cats, q3[column], cats, line_color="black")
        p.segment(lower[column], cats, q1[column], cats, line_color="black")

        # boxes
        p.hbar(cats, 0.7, q2[column], q3[column], fill_color="navy", alpha=0.5, line_color="black")
        p.hbar(cats, 0.7, q1[column], q2[column], fill_color="navy", alpha=0.3, line_color="black")

        # whiskers (almost-0 height rects simpler than segments)
        p.rect(lower[column], cats, 0.01, 0.2, line_color="black")
        p.rect(upper[column], cats, 0.01, 0.2, line_color="black")

        if all_dot:
            p.circle(df[column], df[by], size=4, color="black", fill_alpha=0.3)
        # outliers
        if not out.empty:
            p.circle(outy, outx, size=6, color="#F38630", fill_alpha=1)
            
    if direction == 'vertical':
        # stems
        p.segment(cats, upper[column], cats, q3[column], line_color="black")
        p.segment(cats, lower[column], cats, q1[column], line_color="black")

        # boxes
        p.vbar(cats, 0.7, q2[column], q3[column], fill_color="#E08E79", line_color="black")
        p.vbar(cats, 0.7, q1[column], q2[column], fill_color="#3B8686", line_color="black")

        # whiskers (almost-0 height rects simpler than segments)
        p.rect(cats, lower[column], 0.2, 0.01, line_color="black")
        p.rect(cats, upper[column], 0.2, 0.01, line_color="black")

        if all_dot:
            p.circle(df[by], df[column], size=4, color="grey", fill_alpha=0.6)
        # outliers
        if not out.empty:
            p.circle(outx, outy, size=6, color="#F38630", fill_alpha=1)
        
    

p = figure(title='美国香烟消费 1985～1995', tools="save", 
           # background_fill_color="#efefef", 
           x_range=[30, 215], y_range=[1996, 1984], 
           plot_width=800, plot_height=450, 
           toolbar_location='right', )
p.yaxis.axis_label = '<--- 年 --->'
p.xaxis.axis_label = '香烟消费（平均每人每年的包数）'
p.title.text_font_size = '16pt'
p.axis.axis_label_text_font_size = "12pt"
p.axis.major_label_text_font_size = "12pt"
p.xaxis.ticker = [50, 100, 150, 200]
p.yaxis.ticker = [_ for _ in range(1995, 1984, -1)]
# p.xaxis.ticker.minor_ticks = [51]
# p.ygrid.grid_line_color = "grey"
# p.grid.grid_line_width = 0.5

bokeh_box_horizontal(p, df_cigatette, 'year', 'packpc', direction='horizontal', all_dot=False)

###########################################################################
df, by, column = df_cigatette.copy(), 'year', 'packpc'
groups = df[[column, by]].groupby(by=by, sort=False)
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr
# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group[column] > upper.loc[cat][column]) | (group[column] < lower.loc[cat][column])][column]
out = groups.apply(outliers).dropna()
df['id'] = df.index
df = df.set_index(['year', 'id'])
out.index.values
#df[(1985, 15)]
df.loc[out.index]

###########################################################################
# prepare outlier data for plotting, we need coordinates for every outlier.
outx = []
outy = []
if not out.empty:
    for keys in out.index:
        outx.append(keys[0])
        outy.append(out.loc[keys[0]].loc[keys[1]])
p.circle(outy, outx, size=6, color="#F38630", fill_alpha=1)
p.text(outy, outx, df.loc[out.index]['state'], text_font_size='10px', text_align='center', text_baseline='top', y_offset=5)
show(p)

In [13]:
p.line(x=[100, 100], y=[1996, 1984], line_color='red', line_dash='dashed')
show(p)

---

In [14]:
df_cigatette = pd.read_csv('cigarette.csv', index_col=0)
df_cigatette.head()

,state,year,cpi,pop,packpc,income,tax,avgprs,taxs
1,AL,1985,1.076,3973000,116.486282,46014968,32.500004,102.181671,33.348335
2,AR,1985,1.076,2327000,128.534592,26210736,37.000000,101.474998,37.000000
3,AZ,1985,1.076,3184000,104.522614,43956936,31.000000,108.578751,36.170418
4,CA,1985,1.076,26444000,100.363037,447102816,26.000000,107.837341,32.104000
5,CO,1985,1.076,3209000,112.963539,49466672,31.000000,94.266663,31.000000


 |  mantissas
 |      The acceptable list numbers to generate multiples of.
 |  
 |  max_interval
 |      The largest allowable interval between two adjacent ticks.
 |      
 |      .. note::
 |          To specify an unbounded interval, set to ``None``.
 |  
 |  min_interval
 |      The smallest allowable interval between two adjacent ticks.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from ContinuousTicker:
 |  
 |  desired_num_ticks
 |      A desired target number of major tick positions to generate across
 |      the plot range.
 |      
 |      .. note:
 |          This value is a suggestion, and ticker subclasses may ignore
 |          it entirely, or use it only as an ideal goal to approach as well
 |          as can be, in the context of a specific ticking strategy.
 |  
 |  num_minor_ticks
 |      The number of minor tick positions to generate between
 |      adjacent major tick values.

In [15]:
y_ticker = df_cigatette['state_fn'].unique().tolist()

KeyError: 'state_fn'

In [16]:
p = figure(title='美国香烟消费 1985～1995', tools="save", 
           # background_fill_color="#efefef", 
           x_range=[45, 205], 
           y_range=list(df_cigatette['state_fn'].unique()), 
           plot_width=800, plot_height=1000, 
           toolbar_location='right', )
p.yaxis.axis_label = '<--- 年 --->'
p.xaxis.axis_label = '香烟消费（平均每人每年的包数）'
p.title.text_font_size = '16pt'
p.axis.axis_label_text_font_size = "12pt"
p.axis.major_label_text_font_size = "10pt"
p.xaxis.ticker.min_interval = 50
p.xaxis.ticker.num_minor_ticks = 5
# p.xaxis.ticker = [50, 100, 150, 200]
# p.yaxis.ticker = ['1']
# p.xaxis.ticker.minor_ticks = [51]
# p.ygrid.grid_line_color = "grey"
# p.grid.grid_line_width = 0.5

bokeh_box_horizontal(p, df_cigatette, 'state_fn', 'packpc', direction='horizontal', all_dot=False)
show(p)

KeyError: 'state_fn'

In [17]:
usa_states = '''<table log-set-param="table_view" width="658" class="table-view log-set-param"><tbody><tr><th width="70">缩写</th><th width="155">州名原文</th><th width="134">州名中译</th><th width="152">首府中译</th><th width="146">首府原文</th></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>AL</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Alabama</div>
</td><td width="134" align="left" valign="middle"><a target="_blank" href="/item/%E4%BA%9A%E6%8B%89%E5%B7%B4%E9%A9%AC%E5%B7%9E/7052671" data-lemmaid="7052671">亚拉巴马州</a></td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E8%92%99%E5%93%A5%E9%A9%AC%E5%88%A9/9638033" data-lemmaid="9638033">蒙哥马利</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Montgomery</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>AK</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Alaska</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E9%98%BF%E6%8B%89%E6%96%AF%E5%8A%A0%E5%B7%9E/1947146" data-lemmaid="1947146">阿拉斯加州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%9C%B1%E8%AF%BA/10416013" data-lemmaid="10416013">朱诺</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Juneau</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>AZ</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Arizona</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BA%9A%E5%88%A9%E6%A1%91%E9%82%A3%E5%B7%9E/2016570" data-lemmaid="2016570">亚利桑那州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E8%8F%B2%E5%B0%BC%E5%85%8B%E6%96%AF/4856" data-lemmaid="4856">菲尼克斯</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Phoenix</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>AR</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Arkansas</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E9%98%BF%E8%82%AF%E8%89%B2%E5%B7%9E/3436196" data-lemmaid="3436196">阿肯色州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%B0%8F%E7%9F%B3%E5%9F%8E/5393602" data-lemmaid="5393602">小石城</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Little Rock</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>CA</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">California</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%8A%A0%E5%88%A9%E7%A6%8F%E5%B0%BC%E4%BA%9A%E5%B7%9E/40442" data-lemmaid="40442">加利福尼亚州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E8%90%A8%E5%85%8B%E6%8B%89%E9%97%A8%E6%89%98/5404480" data-lemmaid="5404480">萨克拉门托</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Sacramento</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>CO</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Colorado</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%A7%91%E7%BD%97%E6%8B%89%E5%A4%9A%E5%B7%9E/3288702" data-lemmaid="3288702">科罗拉多州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%B8%B9%E4%BD%9B/2927494" data-lemmaid="2927494">丹佛</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Denver</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>CT</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Connecticut</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%BA%B7%E6%B6%85%E7%8B%84%E6%A0%BC%E5%B7%9E/3436248" data-lemmaid="3436248">康涅狄格州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%93%88%E7%89%B9%E7%A6%8F%E5%BE%B7/2007563" data-lemmaid="2007563">哈特福德</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Hartford</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>DE</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Delaware</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%89%B9%E6%8B%89%E5%8D%8E%E5%B7%9E/3436266" data-lemmaid="3436266">特拉华州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A4%9A%E4%BD%9B/6059172" data-lemmaid="6059172">多佛</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Dover</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>FL</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Florida</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BD%9B%E7%BD%97%E9%87%8C%E8%BE%BE%E5%B7%9E/3436285" data-lemmaid="3436285">佛罗里达州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A1%94%E6%8B%89%E5%93%88%E8%A5%BF/2004734" data-lemmaid="2004734">塔拉哈西</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Tallahassee</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>GA</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Georgia</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BD%90%E6%B2%BB%E4%BA%9A%E5%B7%9E/1706352" data-lemmaid="1706352">佐治亚州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BA%9A%E7%89%B9%E5%85%B0%E5%A4%A7/11926" data-lemmaid="11926">亚特兰大</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Atlanta</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>HI</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Hawaii</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A4%8F%E5%A8%81%E5%A4%B7%E5%B7%9E/3436297" data-lemmaid="3436297">夏威夷州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%81%AB%E5%A5%B4%E9%B2%81%E9%B2%81/81317" data-lemmaid="81317">火奴鲁鲁</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Honolulu</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>ID</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Idaho</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%88%B1%E8%BE%BE%E8%8D%B7%E5%B7%9E/3436312" data-lemmaid="3436312">爱达荷州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%8D%9A%E4%BC%8A%E8%A5%BF/3279458" data-lemmaid="3279458">博伊西</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Boise</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>IL</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Illinois</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BC%8A%E5%88%A9%E8%AF%BA%E4%BC%8A%E5%B7%9E/3437724" data-lemmaid="3437724">伊利诺伊州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%96%AF%E6%99%AE%E6%9E%97%E8%8F%B2%E5%B0%94%E5%BE%B7/1420017" data-lemmaid="1420017">斯普林菲尔德</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Springfield</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>IN</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Indiana</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%8D%B0%E7%AC%AC%E5%AE%89%E7%BA%B3%E5%B7%9E/3437855" data-lemmaid="3437855">印第安纳州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%8D%B0%E7%AC%AC%E5%AE%89%E7%BA%B3%E6%B3%A2%E5%88%A9%E6%96%AF/10473726" data-lemmaid="10473726">印第安纳波利斯</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Indianapolis</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>IA</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Iowa</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E8%89%BE%E5%A5%A5%E7%93%A6%E5%B7%9E/3437927" data-lemmaid="3437927">艾奥瓦州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%BE%97%E6%A2%85%E5%9B%A0/3558622" data-lemmaid="3558622">得梅因</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Des Moines</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>KS</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Kansas</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A0%AA%E8%90%A8%E6%96%AF%E5%B7%9E/2874022" data-lemmaid="2874022">堪萨斯州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%89%98%E7%9A%AE%E5%8D%A1/1162040" data-lemmaid="1162040">托皮卡</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Topeka</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>KY</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Kentucky</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E8%82%AF%E5%A1%94%E5%9F%BA%E5%B7%9E/3438094" data-lemmaid="3438094">肯塔基州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%B3%95%E5%85%B0%E5%85%8B%E7%A6%8F/9682134" data-lemmaid="9682134">法兰克福</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Frankfort</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>LA</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Louisiana</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E8%B7%AF%E6%98%93%E6%96%AF%E5%AE%89%E9%82%A3%E5%B7%9E/2935061" data-lemmaid="2935061">路易斯安那州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%B7%B4%E5%90%9E%E9%B2%81%E6%97%A5/10934876" data-lemmaid="10934876">巴吞鲁日</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Baton Rouge</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>ME</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Maine</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%BC%85%E5%9B%A0%E5%B7%9E/2683845" data-lemmaid="2683845">缅因州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A5%A5%E5%8F%A4%E6%96%AF%E5%A1%94/15920824" data-lemmaid="15920824">奥古斯塔</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Augusta</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>MD</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Maryland</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E9%A9%AC%E9%87%8C%E5%85%B0%E5%B7%9E/3438236" data-lemmaid="3438236">马里兰州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%AE%89%E9%82%A3%E6%B3%A2%E5%88%A9%E6%96%AF/1695308" data-lemmaid="1695308">安那波利斯</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Annapolis</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>MA</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Massachusetts</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E9%A9%AC%E8%90%A8%E8%AF%B8%E5%A1%9E%E5%B7%9E/3020558" data-lemmaid="3020558">马萨诸塞州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%B3%A2%E5%A3%AB%E9%A1%BF/81031" data-lemmaid="81031">波士顿</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Boston</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>MI</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Michigan</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%AF%86%E6%AD%87%E6%A0%B9%E5%B7%9E/3438465" data-lemmaid="3438465">密歇根州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%85%B0%E8%BE%9B/84503" data-lemmaid="84503">兰辛</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Lansing</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>MN</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Minnesota</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%98%8E%E5%B0%BC%E8%8B%8F%E8%BE%BE%E5%B7%9E/181052" data-lemmaid="181052">明尼苏达州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%9C%A3%E4%BF%9D%E7%BD%97/2827142" data-lemmaid="2827142">圣保罗</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">St. Paul</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>MS</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Mississippi</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%AF%86%E8%A5%BF%E8%A5%BF%E6%AF%94%E5%B7%9E/3438634" data-lemmaid="3438634">密西西比州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%9D%B0%E5%85%8B%E9%80%8A/13236831" data-lemmaid="13236831">杰克逊</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Jackson</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>MO</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Missouri</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%AF%86%E8%8B%8F%E9%87%8C%E5%B7%9E/3438672" data-lemmaid="3438672">密苏里州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%9D%B0%E6%96%90%E9%80%8A%E5%9F%8E/2449179" data-lemmaid="2449179">杰斐逊城</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Jefferson City</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>MT</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Montana</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E8%92%99%E5%A4%A7%E6%8B%BF%E5%B7%9E/3438730" data-lemmaid="3438730">蒙大拿州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%B5%B7%E4%BC%A6%E9%82%A3/1943004" data-lemmaid="1943004">海伦那</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Helena</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>NE</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Nebraska</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%86%85%E5%B8%83%E6%8B%89%E6%96%AF%E5%8A%A0%E5%B7%9E/2217761" data-lemmaid="2217761">内布拉斯加州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%9E%97%E8%82%AF%E5%B8%82/4315550" data-lemmaid="4315550">林肯市</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Lincoln</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>NV</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Nevada</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%86%85%E5%8D%8E%E8%BE%BE%E5%B7%9E/2874107" data-lemmaid="2874107">内华达州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%8D%A1%E6%A3%AE%E5%9F%8E/11001834" data-lemmaid="11001834">卡森城</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Carson City</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>NH</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">New Hampshire</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%96%B0%E7%BD%95%E5%B8%83%E4%BB%80%E5%B0%94%E5%B7%9E/3438824" data-lemmaid="3438824">新罕布什尔州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%BA%B7%E7%A7%91%E5%BE%B7/84534" data-lemmaid="84534">康科德</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Concord</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>NJ</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">New Jersey</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%96%B0%E6%B3%BD%E8%A5%BF%E5%B7%9E/3438878" data-lemmaid="3438878">新泽西州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%89%B9%E4%BC%A6%E9%A1%BF/1988236" data-lemmaid="1988236">特伦顿</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Trenton</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>NM</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">New Mexico</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%96%B0%E5%A2%A8%E8%A5%BF%E5%93%A5%E5%B7%9E/3438975" data-lemmaid="3438975">新墨西哥州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%9C%A3%E8%8F%B2/84537" data-lemmaid="84537">圣菲</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Santa Fe</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>NY</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">New York</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%BA%BD%E7%BA%A6%E5%B7%9E/3439049" data-lemmaid="3439049">纽约州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A5%A5%E5%B0%94%E5%B7%B4%E5%B0%BC/84536" data-lemmaid="84536">奥尔巴尼</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Albany</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>NC</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">North Carolina</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%8C%97%E5%8D%A1%E7%BD%97%E6%9D%A5%E7%BA%B3%E5%B7%9E/3439107" data-lemmaid="3439107">北卡罗来纳州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%BD%97%E5%88%A9/16490791" data-lemmaid="16490791">罗利</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Raleigh</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>ND</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">North Dakota</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%8C%97%E8%BE%BE%E7%A7%91%E4%BB%96%E5%B7%9E/3439163" data-lemmaid="3439163">北达科他州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BF%BE%E6%96%AF%E9%BA%A6/2225298" data-lemmaid="2225298">俾斯麦</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Bismarck</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>OH</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Ohio</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BF%84%E4%BA%A5%E4%BF%84%E5%B7%9E/3439219" data-lemmaid="3439219">俄亥俄州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%93%A5%E4%BC%A6%E5%B8%83/9657567" data-lemmaid="9657567">哥伦布</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Columbus</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>OK</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Oklahoma</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BF%84%E5%85%8B%E6%8B%89%E4%BD%95%E9%A9%AC%E5%B7%9E/3439257" data-lemmaid="3439257">俄克拉何马州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BF%84%E5%85%8B%E6%8B%89%E4%BD%95%E9%A9%AC%E5%9F%8E/9863244" data-lemmaid="9863244">俄克拉何马城</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Oklahoma City</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>OR</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Oregon</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BF%84%E5%8B%92%E5%86%88%E5%B7%9E/3439295" data-lemmaid="3439295">俄勒冈州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A1%9E%E5%8B%92%E5%A7%86/18640702" data-lemmaid="18640702">塞勒姆</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Salem</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>PA</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Pennsylvania</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%AE%BE%E5%A4%95%E6%B3%95%E5%B0%BC%E4%BA%9A%E5%B7%9E/5148" data-lemmaid="5148">宾夕法尼亚州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%93%88%E9%87%8C%E6%96%AF%E5%A0%A1/1973648" data-lemmaid="1973648">哈里斯堡</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Harrisburg</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>RI</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Rhode Island</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%BD%97%E5%BE%97%E5%B2%9B%E5%B7%9E/3439332" data-lemmaid="3439332">罗得岛州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%99%AE%E7%BD%97%E7%BB%B4%E7%99%BB%E6%96%AF/2014124" data-lemmaid="2014124">普罗维登斯</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Providence</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>SC</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">South Carolina</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%8D%97%E5%8D%A1%E7%BD%97%E6%9D%A5%E7%BA%B3%E5%B7%9E/7363647" data-lemmaid="7363647">南卡罗来纳州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%93%A5%E4%BC%A6%E6%AF%94%E4%BA%9A/6561398" data-lemmaid="6561398">哥伦比亚</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Columbia</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>SD</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">South Dakota</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%8D%97%E8%BE%BE%E7%A7%91%E4%BB%96%E5%B7%9E/3439373" data-lemmaid="3439373">南达科他州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%9A%AE%E5%B0%94/17490571" data-lemmaid="17490571">皮尔</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Pierre</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>TN</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Tennessee</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%94%B0%E7%BA%B3%E8%A5%BF%E5%B7%9E/3439391" data-lemmaid="3439391">田纳西州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%BA%B3%E4%BB%80%E7%BB%B4%E5%B0%94/26068" data-lemmaid="26068">纳什维尔</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Nashville</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>TX</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Texas</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%BE%97%E5%85%8B%E8%90%A8%E6%96%AF%E5%B7%9E/3439412" data-lemmaid="3439412">得克萨斯州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A5%A5%E6%96%AF%E6%B1%80/15484" data-lemmaid="15484">奥斯汀</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Austin</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>UT</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Utah</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%8A%B9%E4%BB%96%E5%B7%9E/3439429" data-lemmaid="3439429">犹他州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E7%9B%90%E6%B9%96%E5%9F%8E/8808871" data-lemmaid="8808871">盐湖城</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Salt Lake City</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>VT</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Vermont</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E4%BD%9B%E8%92%99%E7%89%B9%E5%B7%9E/3439446" data-lemmaid="3439446">佛蒙特州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E8%92%99%E5%BD%BC%E5%88%A9%E5%9F%83/8939524" data-lemmaid="8939524">蒙彼利埃</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Montpelier</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>VA</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Virginia</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%BC%97%E5%90%89%E5%B0%BC%E4%BA%9A%E5%B7%9E/3439466" data-lemmaid="3439466">弗吉尼亚州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E9%87%8C%E5%A3%AB%E6%BB%A1/964714" data-lemmaid="964714">里士满</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Richmond</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>WA</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Washington</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%8D%8E%E7%9B%9B%E9%A1%BF%E5%B7%9E/2220710" data-lemmaid="2220710">华盛顿州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A5%A5%E6%9E%97%E5%8C%B9%E4%BA%9A/6379216" data-lemmaid="6379216">奥林匹亚</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Olympia</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>WV</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">West Virginia</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E8%A5%BF%E5%BC%97%E5%90%89%E5%B0%BC%E4%BA%9A%E5%B7%9E/2873912" data-lemmaid="2873912">西弗吉尼亚州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%9F%A5%E5%B0%94%E6%96%AF%E9%A1%BF/1761052" data-lemmaid="1761052">查尔斯顿</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Charleston</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>WI</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Wisconsin</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A8%81%E6%96%AF%E5%BA%B7%E6%98%9F%E5%B7%9E/7052558" data-lemmaid="7052558">威斯康星州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E9%BA%A6%E8%BF%AA%E9%80%8A/3692324" data-lemmaid="3692324">麦迪逊</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Madison</div>
</td></tr><tr><td width="70" align="left" valign="middle"><div class="para" label-module="para"><b>WY</b></div>
</td><td width="155" align="left" valign="middle"><div class="para" label-module="para">Wyoming</div>
</td><td width="134" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E6%80%80%E4%BF%84%E6%98%8E%E5%B7%9E/3439543" data-lemmaid="3439543">怀俄明州</a></div>
</td><td width="152" align="left" valign="middle"><div class="para" label-module="para"><a target="_blank" href="/item/%E5%A4%8F%E5%BB%B6/1167577" data-lemmaid="1167577">夏延</a></div>
</td><td width="146" align="left" valign="middle"><div class="para" label-module="para">Cheyenne</div>
</td></tr></tbody></table>'''

In [18]:
import re
agg = re.findall(r'<b>([^<]*)</b>', usa_states)
full = re.findall(r'label-module="para">([^<]*)</div>', usa_states)
full = [full[2*i] for i in range(50)]
zh = re.findall(r'>([^<]*)</a>', usa_states)
zh = [zh[2*i] for i in range(50)]
print(len(agg), len(full), len(zh))

50 50 50


In [19]:
states = dict(zip(agg, [[full[i], zh[i]] for i in range(50)]))

In [20]:
states

{'AL': ['Alabama', '亚拉巴马州'],
 'AK': ['Alaska', '阿拉斯加州'],
 'AZ': ['Arizona', '亚利桑那州'],
 'AR': ['Arkansas', '阿肯色州'],
 'CA': ['California', '加利福尼亚州'],
 'CO': ['Colorado', '科罗拉多州'],
 'CT': ['Connecticut', '康涅狄格州'],
 'DE': ['Delaware', '特拉华州'],
 'FL': ['Florida', '佛罗里达州'],
 'GA': ['Georgia', '佐治亚州'],
 'HI': ['Hawaii', '夏威夷州'],
 'ID': ['Idaho', '爱达荷州'],
 'IL': ['Illinois', '伊利诺伊州'],
 'IN': ['Indiana', '印第安纳州'],
 'IA': ['Iowa', '艾奥瓦州'],
 'KS': ['Kansas', '堪萨斯州'],
 'KY': ['Kentucky', '肯塔基州'],
 'LA': ['Louisiana', '路易斯安那州'],
 'ME': ['Maine', '缅因州'],
 'MD': ['Maryland', '马里兰州'],
 'MA': ['Massachusetts', '马萨诸塞州'],
 'MI': ['Michigan', '密歇根州'],
 'MN': ['Minnesota', '明尼苏达州'],
 'MS': ['Mississippi', '密西西比州'],
 'MO': ['Missouri', '密苏里州'],
 'MT': ['Montana', '蒙大拿州'],
 'NE': ['Nebraska', '内布拉斯加州'],
 'NV': ['Nevada', '内华达州'],
 'NH': ['New Hampshire', '新罕布什尔州'],
 'NJ': ['New Jersey', '新泽西州'],
 'NM': ['New Mexico', '新墨西哥州'],
 'NY': ['New York', '纽约州'],
 'NC': ['North Carolina', '北卡罗来纳州'],
 'ND': ['North Da